In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
import re
import nltk
import pickle
import math

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
# from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

mn = MultinomialNB()
port = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /home/rohan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rohan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rohan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rohan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [25]:
# path ="/content/drive/MyDrive/labelled_ShapeOfYou.csv"
path = "labelled_data_limit_datapoints.csv"
initial_data = pd.read_csv(path,encoding='utf8');
initial_data.head()

,Author,Comment,Polarity
0,Crazy Rabbit,thinking face,0.0
1,ishak laras,Bro trained then ate a fried chicken,-1.0
2,Love Lo fi Mix,Best LOFI Mix song watch my channel best song ...,1.0
3,LizzaMCPE,"5.6B Views Thats like, HALF OF THE WORLD",-1.0
4,filho do dono folded hands,Feat com o chefin ficou otimo..red heartflexed...,0.0


In [26]:
initial_data.shape

(2577, 3)

In [5]:
#make new dataframe for utilizing
# df = initial_data[['Author','Comment','Polarity']].copy()
# df.head()

In [6]:
#convert comments to lower case
# def lowercaseConverter(text):
#     lower_text = [str(text[i]).lower() for i in range(len(text))]
#     return lower_text

In [7]:
#TESTING OUT TOKENIZATION
#sentence tokenizing
#sent_tok = [ sent_tokenize(str(sent)) for sent in df['Comment']]

#word tokenizing
#word_tok = [ word_tokenize(str(sent)) for sent in df['Comment']]

In [27]:
#removing special characters
def specialCharacterRemover(word_tok):
    no_special_char = []
    for words in word_tok:
        clean = []
        for w in words:
            res =re.sub(r'[^\w\s]',"",w)
            if res != "":
                clean.append(res)
        no_special_char.append(clean)
    return no_special_char

In [28]:
#removing stop words
def stopWordsRemover(text):
    no_stopwords = []
    for words in text:
        w=[]
        for word in words:
            if not word in stopwords.words('english'):
                w.append(word)
        no_stopwords.append(w)
    return no_stopwords

In [29]:
#lemmatizing
def lemmatizeWord(text):
    lemmatized_words =[]
    for words in text:
        w=[]
        for word in words:
            a = port.lemmatize(word)
            w.append(a)
        lemmatized_words.append(w)
    return lemmatized_words

In [30]:
def wordListToString(list_words):
    temp=[]
    t=""
    for words in list_words:
        t = " ".join(words)
        temp.append(t)
    return(temp)

In [31]:
X = initial_data['Comment']
y = initial_data['Polarity']

In [32]:
#parameter is a list and return is also a list type
def getCleanedText(text_data):
    #word tokenize
    tokens = [ word_tokenize(str(sent)) for sent in text_data]#lower_text]
    #remove special characters
    no_special = specialCharacterRemover(tokens)
    #remove stop words
    no_stopwords = stopWordsRemover(no_special)
    #lemmatize text
    lemmatized_text = lemmatizeWord(no_stopwords)
    #list of sentences
    clean_text = wordListToString(lemmatized_text)
    return clean_text

In [33]:
# X_clean = getCleanedText(X)
X = getCleanedText(X)

In [34]:
#vectorize
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range =(1,2))#,max_features=25000)

def vectorize_text(list_text):
    return (cv.fit_transform(list_text).toarray())             

In [35]:
#bag of word model
# X_vec = vectorize_text(X_clean)
X = vectorize_text(X)

In [36]:
print("Number of features",len(cv.get_feature_names()))
print(type(X))
print(type(y))

Number of features 23391
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


/home/rohan/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [37]:
#train test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, shuffle=True)
s_f = 0.8
n_train = math.floor(s_f * X.shape[0])
n_test = math.ceil((1-s_f) * X.shape[0])
X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]
print("Total Number of rows in train:",X_train.shape[0])
print("Total Number of rows in test:",X_test.shape[0])

Total Number of rows in train: 2061
Total Number of rows in test: 516


In [38]:
print("Training data points:",len(y_train),"; Testing data points:",len(y_test))

Training data points: 2061 ; Testing data points: 516


In [39]:
#train in multinomial naive bayes classifier
mn.fit(X_train,y_train)

MultinomialNB()

In [40]:
#save trained model
filename = 'finalized_model_limited_datapoints.sav'
pickle.dump(mn,open(filename,'wb'))

In [41]:
# print(len(Xt_vec[0]),len(y_test))
y_pred = mn.predict(X_test)

In [42]:
#convert pandas series to list
y_test = y_test.to_list()

In [43]:
print(y_test[23],y_pred[23])

1.0 1.0


In [44]:
dict ={'label':y_test, 'prediction': y_pred}

In [45]:
final_df = pd.DataFrame(dict)

In [46]:
#save prediction and labels to csv
final_df.to_csv('prediction_limited_datapoints.csv')